<a href="https://colab.research.google.com/github/Marcusleeleelee/FTEC4998-4999/blob/main/FTEC4998_4999.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
# Import necessary packages
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from google.colab import drive
from tqdm import tqdm

# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [56]:
# Step 1: Utils - ok
def uni_list(input): return list(set(input))

In [57]:
class Dataset():
    def __init__(self, file_path):
        self.dataset = pd.read_feather(file_path)
        self.X_train, self.y_train = None, None
        self.X_test, self.y_test = None, None
        self.scalers = None
        self.pca = None
        self.label = 'loan_condition_cat'
        self.original_columns = None

    def show(self, rows=10):
        return self.dataset.head(rows)

    def basic_processing(self):
        temp_func_1 = lambda x: '<=2009' if str(x) in ['2007', '2008', '2009'] else ("[2010, 2012]" if str(x) in ['2010', '2011', '2012'] else '>=2013')
        columns_to_delete = [
            'id', 'issue_d', 'home_ownership_cat', 'income_category', 'income_cat', 'term_cat', 'application_type_cat',
            'purpose_cat', 'interest_payment_cat', 'loan_condition'
        ]
        self.dataset.drop(columns=columns_to_delete, inplace=True)
        self.dataset['grade'] = self.dataset['grade'].apply(temp_func_1)
        self.dataset['final_d'] = self.dataset['final_d'].apply(lambda x: str(x)[-4:]).apply(temp_func_1)
        self.dataset = pd.get_dummies(self.dataset, columns=['year', 'final_d', 'home_ownership', 'term', 'application_type',
                                                             'purpose', 'interest_payments', 'grade', 'region'], dtype=int)

    def train_test_split(self, test_size=0.2, random_state=42):
        X = self.dataset.drop(columns=[self.label])
        y = self.dataset[self.label]
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
        self.original_columns = X.columns

    def preprocessing_train(self):
        scaler = StandardScaler()
        self.X_train = pd.DataFrame(scaler.fit_transform(self.X_train), columns=self.original_columns)
        self.scalers = scaler

        # Perform PCA
        self.pca = PCA(n_components=30)
        pca_components = self.pca.fit_transform(self.X_train)
        self.X_train = pd.DataFrame(pca_components, columns=self.original_columns[:pca_components.shape[1]])

    def preprocessing_test(self):
        # Apply stored scalers
        self.X_test = pd.DataFrame(self.scalers.transform(self.X_test), columns=self.original_columns)

        # Apply PCA
        pca_components = self.pca.transform(self.X_test)
        self.X_test = pd.DataFrame(pca_components, columns=self.original_columns[:pca_components.shape[1]])

In [58]:
# Calculating # ok
data = Dataset('/content/drive/My Drive/Colab Notebooks/FTEC4998_9/loan_final313_processed.feather')
data.basic_processing()
data.train_test_split()
data.preprocessing_train()
data.preprocessing_test()

In [59]:
# Data conversion # ok
train_x, train_y = data.X_train, data.y_train
test_x, test_y = data.X_test, data.y_test
counts = np.mean(train_y == 1) * 100
print(counts)
print(train_x.shape, train_y.shape)
print(test_x.shape, test_y.shape)
# Ensure y_train is binary
assert set(train_y).issubset({0, 1}), "Target values must be 0 or 1 for binary classification."

# Convert to tensors and move to GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
X_train_tensor = torch.tensor(train_x.to_numpy(), dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(train_y.values.ravel(), dtype=torch.float32).unsqueeze(1).to(device)
X_test_tensor = torch.tensor(test_x.to_numpy(), dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(test_y.values.ravel(), dtype=torch.float32).unsqueeze(1).to(device)

7.5910370853482805
(709903, 30) (709903,)
(177476, 30) (177476,)


In [60]:
# Train, predict, and accuracy functions
def train_model(model): # ok
    print("Training")
    model.train()
    for epoch in range(model.epochs):
        model.optimizer.zero_grad()
        outputs = model(model.X_train_tensor)
        loss = model.criterion(outputs, model.y_train_tensor)
        loss.backward()
        model.optimizer.step()
        if epoch % 10 == 0:
            print(f'Epoch {epoch}, Loss: {loss.item()}')


def predict_model(model, X_tensor):
    print('Predicting')
    model.eval()
    with torch.no_grad():
        outputs = model(X_tensor).squeeze()
        return (outputs > 0.5).float()

def calculate_accuracy(model, X_tensor, y_tensor):
    print('Calculating Accuracy.')
    X_tensor = X_tensor.to(next(model.parameters()).device)
    y_tensor = y_tensor.to(next(model.parameters()).device)
    predictions = predict_model(model, X_tensor)

    # Ensure predictions and labels are the same shape
    predictions = predictions.squeeze()
    y_tensor = y_tensor.squeeze()

    correct = (predictions == y_tensor).sum().item()
    accuracy = correct / y_tensor.size(0)
    return accuracy

In [61]:
# Train ANN
# MLP model
class ANN(nn.Module):
    def __init__(self, X_train_tensor, y_train_tensor, lr=0.001):
        super(ANN, self).__init__()
        self.input_dim = X_train_tensor.shape[1]
        self.net = nn.Sequential(
            nn.Linear(self.input_dim, 128),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.LeakyReLU(0.1),
            nn.Dropout(0.3),
            nn.Linear(64, 32),
            nn.BatchNorm1d(32),
            nn.LeakyReLU(0.03),
            nn.Dropout(0.3),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )
        self.criterion = nn.BCELoss()
        self.optimizer = optim.Adam(self.parameters(), lr=lr)
        self.epochs = 100
        self.y_train_tensor = y_train_tensor
        self.X_train_tensor = X_train_tensor

    def forward(self, x):
        return self.net(x)
ann_model = ANN(X_train_tensor, y_train_tensor).to(device)
train_model(ann_model)
ann_predictions = predict_model(ann_model, X_train_tensor).cpu().numpy()
print('ANN train:', calculate_accuracy(ann_model, ann_model.X_train_tensor, ann_model.y_train_tensor))
print('ANN test:', calculate_accuracy(ann_model, X_test_tensor, y_test_tensor))

Training
Epoch 0, Loss: 0.6320598721504211
Epoch 10, Loss: 0.5233052372932434
Epoch 20, Loss: 0.455554723739624
Epoch 30, Loss: 0.40684929490089417
Epoch 40, Loss: 0.369559645652771
Epoch 50, Loss: 0.3382631540298462
Epoch 60, Loss: 0.31156885623931885
Epoch 70, Loss: 0.2894217073917389
Epoch 80, Loss: 0.2711659073829651
Epoch 90, Loss: 0.2562571167945862
Predicting
Calculating Accuracy.
Predicting
ANN train: 0.9461095389088369
Calculating Accuracy.
Predicting
ANN test: 0.9458180260993035


In [62]:
print(ann_predictions)
print(type(ann_predictions))
print(np.unique(ann_predictions))

[0. 0. 0. ... 0. 0. 0.]
<class 'numpy.ndarray'>
[0. 1.]


In [63]:
# Logistic Regression as a neural network
class LogisticRegressionModel(nn.Module):
    def __init__(self, X_train_tensor, y_train_tensor, lr=0.01):
        super(LogisticRegressionModel, self).__init__()
        self.input_dim = X_train_tensor.shape[1]
        self.net = nn.Sequential(
            nn.Linear(self.input_dim, 1),
            nn.Sigmoid()
        )
        self.criterion = nn.BCELoss()
        self.optimizer = optim.Adam(self.parameters(), lr=lr)
        self.epochs = 700
        self.y_train_tensor = y_train_tensor
        self.X_train_tensor = X_train_tensor

    def forward(self, x):
        return self.net(x)
log_reg = LogisticRegressionModel(X_train_tensor, y_train_tensor).to(device)
train_model(log_reg)
log_reg_predictions = predict_model(log_reg, X_train_tensor).cpu().numpy()
print('Logistic Regression train:', calculate_accuracy(log_reg, log_reg.X_train_tensor, log_reg.y_train_tensor))
print('Logistic Regression test:', calculate_accuracy(log_reg, X_test_tensor, y_test_tensor))

Training
Epoch 0, Loss: 0.8041287660598755
Epoch 10, Loss: 0.6881495714187622
Epoch 20, Loss: 0.6203729510307312
Epoch 30, Loss: 0.5704099535942078
Epoch 40, Loss: 0.5297396183013916
Epoch 50, Loss: 0.4944502115249634
Epoch 60, Loss: 0.4639311134815216
Epoch 70, Loss: 0.43751290440559387
Epoch 80, Loss: 0.4144546091556549
Epoch 90, Loss: 0.393987238407135
Epoch 100, Loss: 0.3758220672607422
Epoch 110, Loss: 0.35964682698249817
Epoch 120, Loss: 0.3450913429260254
Epoch 130, Loss: 0.332291841506958
Epoch 140, Loss: 0.32069891691207886
Epoch 150, Loss: 0.3102697730064392
Epoch 160, Loss: 0.3007446825504303
Epoch 170, Loss: 0.29212242364883423
Epoch 180, Loss: 0.2843018174171448
Epoch 190, Loss: 0.27719470858573914
Epoch 200, Loss: 0.27072471380233765
Epoch 210, Loss: 0.26482513546943665
Epoch 220, Loss: 0.2594374120235443
Epoch 230, Loss: 0.2543940544128418
Epoch 240, Loss: 0.24976688623428345
Epoch 250, Loss: 0.24563421308994293
Epoch 260, Loss: 0.24184297025203705
Epoch 270, Loss: 0.238

In [64]:
print(log_reg_predictions)
print(type(log_reg_predictions))
print(np.unique(log_reg_predictions))

[0. 0. 0. ... 0. 0. 0.]
<class 'numpy.ndarray'>
[0. 1.]


In [ ]:
class SVMClassifier():
    def __init__(self, train_x, train_y, fraction=0.1, n_samples=10000000000000):
        # Sample a fraction of the data
        n = min(int(len(train_x) * fraction), n_samples)
        self.X_train = train_x.iloc[:n, :]
        self.y_train = train_y.iloc[:n]
        self.model = None
    def fit(self):
        print("Training.")
        # Use Bagging with SVM
        self.model = BaggingClassifier(
            estimator=SVC(C=0.1, kernel='poly', degree=5, gamma='scale'),
            n_estimators=6,
            random_state=42,
            max_samples= 0.05
        )
        self.model.fit(self.X_train, self.y_train)

    def predict(self, X_test):
        print('Predicting.')
        return self.model.predict(X_test).astype(float)

    def calculate_accuracy(self, X, y):
        print('Calculating Accuracy.')
        predictions = self.predict(X)
        accuracy = accuracy_score(y, predictions)
        return accuracy
SVM_model = SVMClassifier(train_x, train_y)
SVM_model.fit()
SVM_predictions = SVM_model.predict(SVM_model.X_train)
print('SVM train: ', SVM_model.calculate_accuracy(SVM_model.X_train, SVM_model.y_train))
print('SVM test: ', SVM_model.calculate_accuracy(test_x, test_y))

Training.
Predicting.
Calculating Accuracy.
Predicting.
SVM train:  0.9309198478658967
Calculating Accuracy.
Predicting.


In [ ]:
print(SVM_predictions)
print(type(SVM_predictions))
print(np.unique(SVM_predictions))

In [ ]:
class NaiveBayesClassifier:
    def __init__(self, X_train, y_train, priors=None, var_smoothing=1e-9):
        self.model = GaussianNB(priors=priors, var_smoothing=var_smoothing)
        self.X_train = X_train
        self.y_train = y_train

    def fit(self):
        self.model.fit(self.X_train, self.y_train)

    def predict(self, X_test):
        return self.model.predict(X_test).astype(float)

    def calculate_accuracy(self, X, y):
        predictions = self.predict(X)
        accuracy = accuracy_score(y, predictions)
        return accuracy

    def confusion_matrix(self, X, y):
        predictions = self.predict(X)
        return confusion_matrix(y, predictions)

    def classification_report(self, X, y):
        predictions = self.predict(X)
        return classification_report(y, predictions)

# Example usage
NB_model = NaiveBayesClassifier(train_x, train_y)
NB_model.fit()
NB_predictions = NB_model.predict(NB_model.X_train)
print('Naive Bayes train accuracy:', NB_model.calculate_accuracy(NB_model.X_train, NB_model.y_train))
print('Naive Bayes test accuracy:', NB_model.calculate_accuracy(test_x, test_y))

In [ ]:
print(NB_predictions)
print(type(NB_predictions))
print(np.unique(NB_predictions))

In [ ]:
class RandomForestModel():
    def __init__(self, X_train, y_train, n_estimators=10, max_depth=None, random_state=42):
        self.model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=random_state, max_samples = 0.05)
        self.X_train = X_train
        self.y_train = y_train

    def fit(self):
        self.model.fit(self.X_train, self.y_train)

    def predict(self, X_test):
        return self.model.predict(X_test).astype(float)

    def calculate_accuracy(self, X, y):
        predictions = self.predict(X)
        accuracy = accuracy_score(y, predictions)
        return accuracy

    def confusion_matrix(self, X, y):
        predictions = self.predict(X)
        return confusion_matrix(y, predictions)

    def classification_report(self, X, y):
        predictions = self.predict(X)
        return classification_report(y, predictions)

# Example usage
RF_model = RandomForestModel(train_x, train_y)
RF_model.fit()
RF_predictions = RF_model.predict(RF_model.X_train)
print('Random Forest train:', RF_model.calculate_accuracy(RF_model.X_train, RF_model.y_train))
print('Random Forest test:', RF_model.calculate_accuracy(test_x, test_y))

In [ ]:
print(RF_predictions)
print(type(RF_predictions))
print(np.unique(RF_predictions))

In [ ]:
class Adaboost():
    def __init__(self, classes_dict, train_x, train_y):
        self.classes_dict = classes_dict
        self.model_order = classes_dict.keys()
    def extract_data(): pass
adModel = Adaboost(classes_dict={"ANN": ANN, "LR": LogisticRegressionModel, "SVM": SVMClassifier, "NB": NaiveBayesClassifier, "RF": RandomForestModel})